## Part one: centroid but not generating centroids , points attached on edge  

# Creating_Training samples of each year from 2012-2021

In [10]:
import os
import rasterio
import geopandas as gpd
from rasterio.mask import mask

# Define the paths to your files
raster_path = "D:/Project_thesis/Data/Reclassified_new/Reclassified_2021.tif"
shapefiles = {
    'training': './Study_area/training_set.shp',
    'testing': './Study_area/testing_set.shp',
    'validation': './Study_area/validation_set.shp'
}
output_dir = "D:/Project_thesis/Data/Ground_truth"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to clip raster with shapefile
def clip_raster(raster_path, shapefile_path, output_path):
    with rasterio.open(raster_path) as src:
        shapefile = gpd.read_file(shapefile_path)
        shapes = [feature["geometry"] for feature in shapefile.__geo_interface__["features"]]
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

# Clip raster with each shapefile
for key, shapefile_path in shapefiles.items():
    output_path = os.path.join(output_dir, f"Reclassified_2021_{key}.tif")
    clip_raster(raster_path, shapefile_path, output_path)

print("Clipping completed.")


Clipping completed.


In [7]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import os
from collections import defaultdict
import rasterio

# Paths to the input files
files = {
    'training': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_training.tif',
    'testing': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_testing.tif',
    'validation': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_validation.tif'
}

output_dir = "D:/Project_thesis/Data/Ground_truth/Output/"
os.makedirs(output_dir, exist_ok=True)

# Function to read TIFF into numpy array using Rasterio
def read_tiff_to_numpy_rasterio(tiff_path):
    with rasterio.open(tiff_path) as dataset:
        array = dataset.read(1)  # Assuming the class labels are in the first band
        transform = dataset.transform
    return array, transform

# Function to sample pixels
def sample_pixels(array, num_samples=1000):
    sampled_pixels = {}
    
    for label in range(1, 6):  # Classes 1 to 5
        indices = np.column_stack(np.where(array == label))
        if len(indices) < num_samples:
            raise ValueError(f"Not enough pixels for class {label} in the provided raster.")
        
        sampled_indices = indices[np.random.choice(indices.shape[0], num_samples, replace=False)]
        sampled_pixels[label] = sampled_indices
    
    return sampled_pixels

# Function to compute centroids from sampled pixels
def compute_centroids(sampled_pixels, transform):
    centroids = []
    for class_value, pixels in sampled_pixels.items():
        points = []
        for pixel in pixels:
            x, y = rasterio.transform.xy(transform, pixel[0], pixel[1])
            points.append((x, y))
        points_array = np.array(points)
        centroid_x = np.mean(points_array[:, 0])
        centroid_y = np.mean(points_array[:, 1])
        centroids.append({'class': class_value, 'geometry': Point(centroid_x, centroid_y)})
    return centroids

# Process each file
for key, file_path in files.items():
    array, transform = read_tiff_to_numpy_rasterio(file_path)
    sampled_pixels = sample_pixels(array)
    
    centroids = compute_centroids(sampled_pixels, transform)
    
    # Save centroids to shapefile
    gdf = gpd.GeoDataFrame(centroids, crs="EPSG:4326")  # Change CRS as needed
    output_shapefile = os.path.join(output_dir, f"{key}_centroids.shp")
    gdf.to_file(output_shapefile, driver="ESRI Shapefile")

print("Processing and centroid computation completed.")


Processing and centroid computation completed.


## Step 1: Read the TIFF files

In [16]:
import os
import numpy as np
import glob
from osgeo import gdal, ogr, osr

def read_tiff_to_numpy_gdal(tiff_path):
    dataset = gdal.Open(tiff_path)
    band = dataset.GetRasterBand(1)  # Assuming class labels are in the first band
    array = band.ReadAsArray()
    return array, dataset.GetGeoTransform()


## Step 2: Sample 1000 pixels per class (1 to 5) for each file

In [17]:
def sample_pixels(array, num_samples=1000):
    unique_classes = np.unique(array)
    sampled_pixels = {label: None for label in unique_classes if label in [1, 2, 3, 4, 5]}
    
    for label in sampled_pixels.keys():
        indices = np.where(array == label)
        sampled_indices = np.random.choice(range(len(indices[0])), num_samples, replace=False)
        sampled_pixels[label] = (indices[0][sampled_indices], indices[1][sampled_indices])
    
    return sampled_pixels


### Step 3: Convert pixel indices to geographic coordinates

In [21]:
def pixel_to_coord(x, y, geotransform):
    x_geo = geotransform[0] + geotransform[1] * x + geotransform[2] * y
    y_geo = geotransform[3] + geotransform[4] * x + geotransform[5] * y
    return (x_geo, y_geo)


### Step 4: Create shapefile from selected pixels

In [25]:
def create_shapefile_from_selected_pixels(selected_pixels, shapefile_path, geotransform):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(shapefile_path):
        driver.DeleteDataSource(shapefile_path)
    data_source = driver.CreateDataSource(shapefile_path)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    
    layer = data_source.CreateLayer(shapefile_path, srs, ogr.wkbPoint)
    layer.CreateField(ogr.FieldDefn("ID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassName", ogr.OFTString))
    
    class_names = {1: "Antropic", 2: "Primary forest", 3: "Secondary forest", 4: "Deforestation", 5: "Water"}
    
    point_id = 1  # Initialize point ID counter
    
    for class_label, (y_indices, x_indices) in selected_pixels.items():
        class_name = class_names.get(class_label, "Unknown")
        for y, x in zip(y_indices, x_indices):
            x_geo, y_geo = pixel_to_coord(x, y, geotransform)
            
            feature = ogr.Feature(layer.GetLayerDefn())
            feature.SetField("ID", point_id)  # Set the point ID
            feature.SetField("ClassID", int(class_label))
            feature.SetField("ClassName", class_name)
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(x_geo, y_geo)
            feature.SetGeometry(point)
            layer.CreateFeature(feature)
            
            point_id += 1  # Increment point ID for the next feature
            
            feature = None  # Clean up
    
    data_source = None
    print(f"Shapefile created: {shapefile_path}")


In [27]:
# New structure for saving shapefiles
base_shapefiles_path = "D:/Project_thesis/Data/Ground_truth/Output/"

if not os.path.exists(base_shapefiles_path):
    os.makedirs(base_shapefiles_path)

image_types = ['training', 'testing', 'validation']

for image_type in image_types:
    # Determine the appropriate subdirectory for shapefiles based on image type
    shapefiles_sub_path = os.path.join(base_shapefiles_path, image_type)
    if not os.path.exists(shapefiles_sub_path):
        os.makedirs(shapefiles_sub_path)
    
    pattern = f'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_{image_type}.tif'
    
    array, geotransform = read_tiff_to_numpy_gdal(pattern)
    sampled_pixels = sample_pixels(array)
    # Naming: e.g., "training_2021.shp"
    shapefile_name = f"{image_type}_2021.shp"
    shapefile_path = os.path.join(shapefiles_sub_path, shapefile_name)
    create_shapefile_from_selected_pixels(sampled_pixels, shapefile_path, geotransform)


Shapefile created: D:/Project_thesis/Data/Ground_truth/Output/training\training_2021.shp
Shapefile created: D:/Project_thesis/Data/Ground_truth/Output/testing\testing_2021.shp
Shapefile created: D:/Project_thesis/Data/Ground_truth/Output/validation\validation_2021.shp


## part two code 

## Step 1: Read the TIFF files

In [70]:
import os
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.geometry import Point
from sklearn.cluster import KMeans
import warnings

def read_tiff_to_numpy_gdal(tiff_path):
    dataset = gdal.Open(tiff_path)
    band = dataset.GetRasterBand(1)  # Assuming class labels are in the first band
    array = band.ReadAsArray()
    return array, dataset.GetGeoTransform()


## Step 2: Sample 1000 pixels per class (1 to 5)

In [78]:
def sample_pixels(array, num_samples=100):
    unique_classes = np.unique(array)
    sampled_pixels = {label: None for label in unique_classes if label in [1, 2, 3, 4, 5]}
    
    for label in sampled_pixels.keys():
        indices = np.where(array == label)
        if len(indices[0]) < num_samples:
            raise ValueError(f"Not enough pixels for class {label} in the provided raster.")
        sampled_indices = np.random.choice(range(len(indices[0])), num_samples, replace=False)
        sampled_pixels[label] = (indices[0][sampled_indices], indices[1][sampled_indices])
    
    return sampled_pixels


## Step 3: Convert pixel indices to geographic coordinates

In [79]:
def pixel_to_coord(x, y, geotransform):
    x_geo = geotransform[0] + geotransform[1] * (x + 0.5) + geotransform[2] * (y + 0.5)
    y_geo = geotransform[3] + geotransform[4] * (x + 0.5) + geotransform[5] * (y + 0.5)
    return (x_geo, y_geo)


## Step 4: Cluster sampled pixels and compute centroids

In [80]:
def cluster_pixels_and_compute_centroids(sampled_pixels, geotransform, num_clusters=1000):
    centroids = []
    point_id = 1
    class_names = {1: "Antropic", 2: "Primary forest", 3: "Secondary forest", 4: "Deforestation", 5: "Water"}
    
    for class_value, (y_indices, x_indices) in sampled_pixels.items():
        points = [pixel_to_coord(x, y, geotransform) for y, x in zip(y_indices, x_indices)]
        points_array = np.array(points)
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
            kmeans.fit(points_array)
        
        cluster_centers = kmeans.cluster_centers_
        
        for center in cluster_centers:
            centroids.append({
                'ID': point_id,
                'class': int(class_value),
                'ClassName': class_names[class_value],
                'geometry': Point(center[0], center[1])
            })
            point_id += 1
    
    return centroids


## Step 5: Create shapefile from centroids

In [81]:
def create_shapefile_from_centroids(centroids, shapefile_path):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver is not available.")
    
    if os.path.exists(shapefile_path):
        driver.DeleteDataSource(shapefile_path)
    
    data_source = driver.CreateDataSource(shapefile_path)
    if data_source is None:
        raise RuntimeError(f"Failed to create data source: {shapefile_path}")
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    
    layer = data_source.CreateLayer('', srs, ogr.wkbPoint)
    if layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile: {shapefile_path}")
    
    layer.CreateField(ogr.FieldDefn("ID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassName", ogr.OFTString))
    
    for centroid in centroids:
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("ID", centroid['ID'])  # Set the point ID
        feature.SetField("ClassID", int(centroid['class']))
        feature.SetField("ClassName", centroid['ClassName'])
        
        geometry = centroid['geometry']
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(geometry.x, geometry.y)
        feature.SetGeometry(point)
        layer.CreateFeature(feature)
        
        feature = None  # Clean up
    
    data_source = None
    print(f"Shapefile created: {shapefile_path}")


## Step 6: Process each TIFF file

In [84]:
# Paths to the input files
files = {
    'training': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_training.tif',
    'testing': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_testing.tif',
    'validation': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_validation.tif'
}

output_dir = "D:/Project_thesis/Data/Ground_truth/Output500/"
os.makedirs(output_dir, exist_ok=True)

for key, file_path in files.items():
    array, geotransform = read_tiff_to_numpy_gdal(file_path)
    sampled_pixels = sample_pixels(array)
    
    centroids = cluster_pixels_and_compute_centroids(sampled_pixels, geotransform)
    
    # Save centroids to shapefile
    shapefile_name = f"{key}_centroids.shp"
    shapefile_path = os.path.join(output_dir, shapefile_name)
    create_shapefile_from_centroids(centroids, shapefile_path)

print("Processing and centroid computation completed.")


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/training_centroids.shp


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/testing_centroids.shp


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/validation_centroids.shp
Processing and centroid computation completed.


In [ ]:
def create_shapefile_from_centroids(centroids, shapefile_path):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver is not available.")
    
    if os.path.exists(shapefile_path):
        driver.DeleteDataSource(shapefile_path)
    
    data_source = driver.CreateDataSource(shapefile_path)
    if data_source is None:
        raise RuntimeError(f"Failed to create data source: {shapefile_path}")
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    
    layer = data_source.CreateLayer('', srs, ogr.wkbPoint)
    if layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile: {shapefile_path}")
    
    layer.CreateField(ogr.FieldDefn("ID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassName", ogr.OFTString))
    
    for centroid in centroids:
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("ID", centroid['ID'])  # Set the point ID
        feature.SetField("ClassID", int(centroid['class']))
        feature.SetField("ClassName", centroid['ClassName'])
        
        geometry = centroid['geometry']
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(geometry.x, geometry.y)
        feature.SetGeometry(point)
        layer.CreateFeature(feature)
        
        feature = None  # Clean up
    
    data_source = None
    print(f"Shapefile created: {shapefile_path}")

# Paths to the input files
files = {
    'training': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_training.tif',
    'testing': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_testing.tif',
    'validation': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_validation.tif'
}

output_dir = "D:/Project_thesis/Data/Ground_truth/Output/"
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories if they don't exist
for subdir in files.keys():
    os.makedirs(os.path.join(output_dir, subdir), exist_ok=True)

for key, file_path in files.items():
    array, geotransform = read_tiff_to_numpy_gdal(file_path)
    sampled_pixels = sample_pixels(array)
    
    centroids = cluster_pixels_and_compute_centroids(sampled_pixels, geotransform)
    
    # Save centroids to shapefile
    shapefile_name = f"{key}_centroids.shp"
    shapefile_path = os.path.join(output_dir, key, shapefile_name)
    create_shapefile_from_centroids(centroids, shapefile_path)

print("Processing and centroid computation completed.")

In [83]:
import os
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.geometry import Point
from sklearn.cluster import KMeans
import warnings

def read_tiff_to_numpy_gdal(tiff_path):
    dataset = gdal.Open(tiff_path)
    band = dataset.GetRasterBand(1)  # Assuming class labels are in the first band
    array = band.ReadAsArray()
    return array, dataset.GetGeoTransform()

def sample_pixels(array, num_samples=500):
    unique_classes = np.unique(array)
    sampled_pixels = {label: None for label in unique_classes if label in [1, 2, 3, 4, 5]}
    
    for label in sampled_pixels.keys():
        indices = np.where(array == label)
        if len(indices[0]) < num_samples:
            raise ValueError(f"Not enough pixels for class {label} in the provided raster.")
        sampled_indices = np.random.choice(range(len(indices[0])), num_samples, replace=False)
        sampled_pixels[label] = (indices[0][sampled_indices], indices[1][sampled_indices])
    
    return sampled_pixels

def pixel_to_coord(x, y, geotransform):
    x_geo = geotransform[0] + geotransform[1] * (x + 0.5) + geotransform[2] * (y + 0.5)
    y_geo = geotransform[3] + geotransform[4] * (x + 0.5) + geotransform[5] * (y + 0.5)
    return (x_geo, y_geo)

def cluster_pixels_and_compute_centroids(sampled_pixels, geotransform, num_clusters=500):
    centroids = []
    point_id = 1
    class_names = {1: "Antropic", 2: "Primary forest", 3: "Secondary forest", 4: "Deforestation", 5: "Water"}
    
    for class_value, (y_indices, x_indices) in sampled_pixels.items():
        points = [pixel_to_coord(x, y, geotransform) for y, x in zip(y_indices, x_indices)]
        points_array = np.array(points)
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
            kmeans.fit(points_array)
        
        cluster_centers = kmeans.cluster_centers_
        
        for center in cluster_centers:
            centroids.append({
                'ID': point_id,
                'class': int(class_value),
                'ClassName': class_names[class_value],
                'geometry': Point(center[0], center[1])
            })
            point_id += 1
    
    return centroids

def create_shapefile_from_centroids(centroids, shapefile_path):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver is not available.")
    
    if os.path.exists(shapefile_path):
        driver.DeleteDataSource(shapefile_path)
    
    data_source = driver.CreateDataSource(shapefile_path)
    if data_source is None:
        raise RuntimeError(f"Failed to create data source: {shapefile_path}")
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    
    layer = data_source.CreateLayer('', srs, ogr.wkbPoint)
    if layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile: {shapefile_path}")
    
    layer.CreateField(ogr.FieldDefn("ID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassID", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("ClassName", ogr.OFTString))
    
    for centroid in centroids:
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("ID", centroid['ID'])  # Set the point ID
        feature.SetField("ClassID", int(centroid['class']))
        feature.SetField("ClassName", centroid['ClassName'])
        
        geometry = centroid['geometry']
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(geometry.x, geometry.y)
        feature.SetGeometry(point)
        layer.CreateFeature(feature)
        
        feature = None  # Clean up
    
    data_source = None
    print(f"Shapefile created: {shapefile_path}")

# Paths to the input files
files = {
    'training': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_training.tif',
    'testing': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_testing.tif',
    'validation': 'D:/Project_thesis/Data/Ground_truth/Reclassified_2021_validation.tif'
}

output_dir = "D:/Project_thesis/Data/Ground_truth/Output500/"
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories if they don't exist
for subdir in files.keys():
    os.makedirs(os.path.join(output_dir, subdir), exist_ok=True)

for key, file_path in files.items():
    array, geotransform = read_tiff_to_numpy_gdal(file_path)
    sampled_pixels = sample_pixels(array)
    
    centroids = cluster_pixels_and_compute_centroids(sampled_pixels, geotransform)
    
    # Save centroids to shapefile
    shapefile_name = f"{key}_centroids.shp"
    shapefile_path = os.path.join(output_dir, key, shapefile_name)
    create_shapefile_from_centroids(centroids, shapefile_path)

print("Processing and centroid computation completed.")


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/training\training_centroids.shp


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/testing\testing_centroids.shp


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows wi

Shapefile created: D:/Project_thesis/Data/Ground_truth/Output500/validation\validation_centroids.shp
Processing and centroid computation completed.


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
